In [10]:
import torch
import torchvision
import torchvision.transforms.v2 as v2
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm
from torch.utils.data import random_split


In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [12]:
transform = v2.Compose([
    v2.ToTensor(),
    v2.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
])
train_transform = v2.Compose([
    v2.RandomHorizontalFlip(),  # Кездейсоқ айналдыру
    v2.RandomRotation(10),  # -10° +10° аралығында бұру
    v2.RandomCrop(32, padding=4),  # 32x32-ге кездейсоқ кесу
    v2.ToTensor(),
    v2.Normalize(mean=[0.5], std=[0.5])
])


C:\Users\abobantai\AppData\Roaming\Python\Python39\site-packages\torchvision\transforms\v2\_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [13]:
trainset = torchvision.datasets.CIFAR10(root='C:/Users/abobantai/Desktop/IT/CIFAR-10_PyTorch', download=True, train=True, transform=train_transform)
train_data, val_data = random_split(trainset, [0.8, 0.2])
train_data = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
val_data = torch.utils.data.DataLoader(val_data, shuffle=True, batch_size= 64)
val_data.dataset.transform = transform
testset = torchvision.datasets.CIFAR10(root='C:/Users/abobantai/Desktop/IT/CIFAR-10_PyTorch', download=True, train=False, transform=transform)
test_data = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [14]:
class MyModel(nn.Module):
    def __init__(self, in_channels, output):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, 32, 3, bias=True),
            nn.ReLU(),
            nn.MaxPool2d(stride=2, kernel_size=2),
            nn.Conv2d(32, 64, 3, bias=True),
            nn.ReLU(),
            nn.MaxPool2d(stride=2, kernel_size=2),
        )
        self.flat = nn.Flatten()
        self.fc = nn.Sequential(
            nn.Linear(2304, 512),
            nn.ReLU(),
            nn.Dropout(0.7),
            nn.Linear(512, 10)
        )
    def forward(self, x):
        x = self.conv(x)
        x = self.flat(x)
        x = self.fc(x)
        return x
model = MyModel(3, 10).to(device)


In [15]:
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
loss_model = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1,patience=4)

In [16]:
epochs = 50
count_without_best_metrics = 0
train_loss = []
train_acc = []
val_loss = []
val_acc = []
lr_list = []
best_loss = None


In [17]:
for epoch in range(epochs):
    train_loop = tqdm(train_data, leave=False)
    model.train()
    true_answers = 0
    total_answers = 0
    running_train_loss = []
    for x, target in train_loop:
        x = x.to(device)
        target = target.reshape(-1).to(torch.int32) # преоброзавание данных для модели\n",
        target = torch.eye(10)[target].to(device) # преоброзавание правильных ответов на ванхотвектора\n",
        pred = model(x)
        loss = loss_model(pred, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        true_answers += (pred.argmax(dim=1) == target.argmax(dim=1)).sum().item()
        total_answers += len(x)
        running_train_loss.append(loss.item())
        mean_train_loss = sum(running_train_loss) / len(running_train_loss)
        mean_train_acc= true_answers / total_answers
        train_loop.set_description(f"EPOCH: [{epoch+1}], train_loss: [{mean_train_loss:.4f}], train_acc: [{mean_train_acc:.4f}], without_best_metrics: [{count_without_best_metrics}]")
    train_loss.append(mean_train_loss)
    train_acc.append(mean_train_acc)
    model.eval()
    with torch.no_grad():
        true_answers = 0
        total_answers = 0
        running_val_loss = []
        for x, target in val_data:
            x = x.to(device)
            target = target.reshape(-1).to(torch.int32) # преоброзавание данных для модели\n",
            target = torch.eye(10)[target].to(device) # преоброзавание правильных ответов на ванхотвектора\n",
            pred = model(x)
            loss = loss_model(pred, target)
            true_answers += (pred.argmax(dim=1) == target.argmax(dim=1)).sum().item()
            total_answers += len(x)
            running_val_loss.append(loss.item())
            mean_val_loss = sum(running_val_loss) / len(running_val_loss)
            mean_val_acc= true_answers / total_answers
        val_loss.append(mean_val_loss)
        val_acc.append(mean_val_acc)
    scheduler.step(mean_val_loss)
    lr = scheduler._last_lr[0]
    print(f"epoch [{epoch+1}/{epochs}], train_loss = {mean_train_loss:.4f}, train_acc = {mean_train_acc:.4f}, val_loss = {mean_val_loss:.4f}, val_acc = {mean_val_acc:.4f}, lr = {lr}")

    if best_loss == None:
        best_loss = mean_val_loss
    if count_without_best_metrics > 10:
        print(f'mertvaya tochka epoch: [{epoch}]')
        break
    if mean_val_loss < best_loss:
        best_loss = mean_val_loss
        count_without_best_metrics = 0
        checkpoint = {
            "state_model": model.state_dict(),
            "state_optimizer": optimizer.state_dict(),
            "state_scheduler": scheduler.state_dict(),
            "loss": {
                "train_loss": train_loss,
                "val_loss": val_loss,
                "best_loss": best_loss
            },
            "metric":{
                "train_accuracy": train_acc,
                "val_accuracy": val_acc
            },
            "lr": lr_list,
            "epoch": {
                "epochs": epochs,
                "save_epochs": epoch
            }
        }
        torch.save(checkpoint, 'checkpoints/model_state_dict.pt')
        print(f"checkpoint on epoch {epoch+1} with best val metrics: {mean_val_loss:.6f}")
    count_without_best_metrics += 1





epoch [1/50], train_loss = 1.7383, train_acc = 0.3548, val_loss = 1.4993, val_acc = 0.4494, lr = 0.001


epoch [2/50], train_loss = 1.4835, train_acc = 0.4593, val_loss = 1.3247, val_acc = 0.5205, lr = 0.001
checkpoint on epoch 2 with best val metrics: 1.324748


epoch [3/50], train_loss = 1.3823, train_acc = 0.5004, val_loss = 1.2877, val_acc = 0.5424, lr = 0.001
checkpoint on epoch 3 with best val metrics: 1.287681


epoch [4/50], train_loss = 1.3139, train_acc = 0.5256, val_loss = 1.1884, val_acc = 0.5795, lr = 0.001
checkpoint on epoch 4 with best val metrics: 1.188360


epoch [5/50], train_loss = 1.2665, train_acc = 0.5466, val_loss = 1.1514, val_acc = 0.5872, lr = 0.001
checkpoint on epoch 5 with best val metrics: 1.151416


...

epoch [46/50], train_loss = 0.9021, train_acc = 0.6855, val_loss = 0.8140, val_acc = 0.7136, lr = 0.001


epoch [47/50], train_loss = 0.9050, train_acc = 0.6843, val_loss = 0.8135, val_acc = 0.7221, lr = 0.001


epoch [48/50], train_loss = 0.9071, train_acc = 0.6855, val_loss = 0.8336, val_acc = 0.7141, lr = 0.001


epoch [49/50], train_loss = 0.8912, train_acc = 0.6904, val_loss = 0.8082, val_acc = 0.7200, lr = 0.001
checkpoint on epoch 49 with best val metrics: 0.808227


epoch [50/50], train_loss = 0.8936, train_acc = 0.6885, val_loss = 0.8273, val_acc = 0.7095, lr = 0.001
